In [ ]:
"""
To 


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import datajoint_utils as du

INFO - 2021-02-07 22:18:35,766 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-07 22:18:35,768 - settings - Setting database.user to celiib
INFO - 2021-02-07 22:18:35,769 - settings - Setting database.password to newceliipass
INFO - 2021-02-07 22:18:35,781 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-07 22:18:35,781 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-07 22:18:35,795 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-07 22:18:36,068 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-07 22:18:36,172 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-07 22:18:36,462 - settings - Setting enable_python_native_blobs to True


# Inspecting Decimation

In [ ]:
minnie.MultiSomaProofread2()

In [ ]:
minnie.MultiSomaProofread2.proj()

In [ ]:
minnie.MultiSomaProofread2.proj() - minnie.Mesh.proj()

In [ ]:
minnie.Decimation() & minnie.MultiSomaProofread2.proj()

In [ ]:
du.get_seg_extracted_somas(864691134947393276)

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
minnie.NeuronGliaNuclei()

In [ ]:
curr_table = (schema.jobs & "table_name='__baylor_segment_centroid'")# & "status='error'"#.delete()
curr_table

In [ ]:
(curr_table & "key_hash = '8de77354e83e5c8764c3791b865c7033'").delete()#.fetch1("key")

In [ ]:
print((curr_table & "key_hash = '3415f98d1d6285f5a403a6b83ddae5ef'").fetch1("error_stack"))

In [ ]:
minnie,schema = du.configure_minnie_vm()

# Inspecting Soma Extraction

In [ ]:
import datajoint as dj
segment_id = (dj.U("segment_id") & minnie.BaylorSegmentCentroid.proj() & minnie.MultiSomaProofread2.proj()).fetch("segment_id")
len(segment_id)

In [ ]:
curr_index = 53
curr_segment_id = segment_id[curr_index]
curr_segment_id

In [ ]:
curr_table = minnie.BaylorSegmentCentroid() & dict(segment_id=curr_segment_id)
curr_table

In [ ]:
minnie.NeuronGliaNuclei() & dict(segment_id=curr_segment_id)

In [ ]:
minnie.MultiSomaProofread2 & dict(segment_id=curr_segment_id)

In [ ]:
import neuron_visualizations as nviz
nviz.plot_objects(meshes=du.get_seg_extracted_somas(curr_segment_id),
                 meshes_colors="random")

In [ ]:
du.plot_decimated_mesh_with_somas(curr_segment_id,
                                 plot_glia_nuclei=False)

In [ ]:
import system_utils as su

In [ ]:
su.decompress_pickle

# Inspecting Decomposition

In [ ]:
len(minnie.Decomposition())

In [ ]:
minnie.Decomposition

In [ ]:
minnie.Decomposition() & "n_error_limbs>3"

In [ ]:
schema.external['decomposition'].delete(delete_external_files=True)

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
error_table = (curr_table & "status='error'")#.delete()
error_table

In [ ]:
print((error_table & "key_hash='2fce6f814c2680709bb844fc65ac2a5b'").fetch1("key"))

In [ ]:
error_table = ((minnie.schema.jobs & "table_name='__decomposition'") & "status='error'")#.delete()

# Checking the Splitting

In [ ]:
(schema.jobs & "table_name = '__decomposition_split'" & "status='error'")

In [ ]:
print((schema.jobs & "table_name = '__decomposition_split'" & "status='error'" & "key_hash='60adf6d2c754098d0cfa6c7cee7bbf21'").fetch1("error_stack"))

In [ ]:
schema.jobs & "table_name = '__decomposition_split'"

In [ ]:
import datajoint as dj
len(dj.U("segment_id") & minnie.DecompositionSplit())

In [ ]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0"
len(key_source)

In [ ]:
minnie.DecompositionSplit() & "n_limbs>6" & "multiplicity > 1"

In [ ]:
neuron_objs = (minnie.DecompositionSplit() & "segment_id=864691134884798458").fetch("decomposition")

In [ ]:
neuron_objs

In [ ]:
import proofreading_utils as pru
import classification_utils as clu

# Developing the Dendrite to Axon Flip Function

In [ ]:
current_axon_limb_branch_dict = clu.axon_like_limb_branch_dict(neuron_objs[0],
                                                              plot_axon_like=True)

In [ ]:
current_axon_limb_branch_dict = clu.axon_like_limb_branch_dict(neuron_objs[1],
                                                              plot_axon_like=True)

# Testing the Neuron

In [ ]:
clu.axon_classification(neuron_objs[2],
                        plot_axon_like_segments=False,
                        plot_candidates=False,
                       plot_axons=True,
                       verbose=True)

In [ ]:
print("hi")

In [ ]:
output_list = pru.proofread_neuron(
    
    input_neuron = neuron_objs[1],

    attempt_to_split_neuron = False,
    plot_neuron_split_results = False,

    plot_neuron_before_filtering = False,

    plot_axon = False,
    plot_axon_like = False,

    # -- for the filtering loop
    plot_limb_branch_filter_with_disconnect_effect = True,
    plot_final_filtered_neuron = True,

    # -- for the output --
    return_process_info = True,

    debug_time = True,
    verbose = True,
    verbose_outline=True

    )

In [ ]:
nru.low_branch_length_clusters(neuron_objs[2],
                              verbose=True)

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
minnie.NeuronSplitSuggestions()

In [ ]:
(schema.jobs & "table_name = '__neuron_split_suggestions'")#.delete()

In [ ]:
minnie.NeuronSplitSuggestions.fetch1("split_results")

# Saving off the split results (becuase redoing)

In [ ]:
split_info = minnie.NeuronSplitSuggestions.fetch(as_dict=True)

In [ ]:
import pandas as pd
split_table = pd.DataFrame.from_dict(split_info)
split_table

In [ ]:
import system_utils as su
su.compressed_pickle(split_table,"split_table")

# Saving and Clearing all of the tables so we can redo

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
col_to_keep_soma = [           
"centroid_x",
"centroid_y",
"centroid_z",
"n_vertices",
"n_faces",
"multiplicity",
"sdf",
"max_side_ratio",
"bbox_volume_ratio",
"max_hole_length",
"run_time",
              ]
minnie.BaylorSegmentCentroid.proj(*col_to_keep_soma)

In [ ]:
soma_saved_data = minnie.BaylorSegmentCentroid.proj(*col_to_keep_soma).fetch(as_dict=True)
new_data_frame_soma = pd.DataFrame.from_dict(soma_saved_data)
new_data_frame_soma

In [ ]:
su.compressed_pickle(new_data_frame_soma,"BaylorSegmentCentroid_saved_1_9")

In [ ]:
minnie.BaylorSegmentCentroid().delete()

In [ ]:
schema.external['somas'].delete(delete_external_files=True)

In [ ]:
(schema.jobs & "table_name='__baylor_segment_centroid'")

In [ ]:
(schema.jobs & "table_name='__baylor_segment_centroid'") & 'error_message ="TypeError: object of type \'Trimesh\' has no len()"'

In [ ]:
nu.all_unique_choose_2_combinations(np.arange(5))

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
(schema.jobs & "table_name='__baylor_segment_centroid'") #& "status='error'"#.delete()

In [ ]:
print((schema.jobs & "table_name='__baylor_segment_centroid'" & 'key_hash="00471f14c125b842baffaf6ebff1eed1"').fetch1("error_stack"))

# Getting a histogram of the somas

In [ ]:
import matplotlib.pyplot as plt
soma_faces_array = new_data_frame_soma["n_faces"].to_numpy()
soma_faces_array_filtered = soma_faces_array[soma_faces_array!=0]
soma_faces_array_filtered

In [ ]:
plt.hist(soma_faces_array_filtered,bins=100)

In [ ]:
import numpy_utils as nu

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
minnie.NeuronGliaNuclei()

In [ ]:
curr_seg_id = 864691134947393276

In [ ]:
nuclei_faces = (minnie.NeuronGliaNuclei() & f"segment_id={curr_seg_id}").fetch1("nuclei_faces")

In [ ]:
current_neuron_mesh = du.fetch_segment_id_mesh(864691134947393276)

In [ ]:
nuclei_mesh = current_neuron_mesh.submesh([nuclei_faces],append=True,repair=False)
nuclei_mesh

In [ ]:
nviz.plot_objects(nuclei_mesh)

In [ ]:
soma_meshes = np.array(du.get_seg_extracted_somas(curr_seg_id))
soma_meshes

In [ ]:
nviz.plot_objects(meshes=soma_meshes[3:4])

In [ ]:
minnie.NeuronGliaNuclei()

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
minnie.BaylorSegmentCentroid

In [ ]:
minnie.NeuronGliaNuclei()

In [ ]:
from importlib import reload
du = reload(du)

In [ ]:
du.plot_decimated_mesh_with_somas(864691135012398582)

# Saving off the nuclei and glia information

In [ ]:
glia_nuclei_info = minnie.NeuronGliaNuclei.fetch(as_dict=True)

In [ ]:
glia_nuclei_info

In [ ]:
du.save_glia_nuclei_files?

In [ ]:
# Now do the table insert
import copy
import datajoint_utils as du
keys_to_insert = []
for k in glia_nuclei_info:
    g_path, n_path = du.save_glia_nuclei_files(k["glia_faces"],
                             k["nuclei_faces"],
                             k["segment_id"])
    new_key = copy.deepcopy(k)
    new_key["glia_faces"] = g_path
    new_key["nuclei_faces"] = n_path
    keys_to_insert.append(new_key)
    

In [ ]:
#so that it will have the adapter defined
from datajoint_utils import *

minnie.NeuronGliaNuclei.insert(keys_to_insert)

In [ ]:
minnie.NeuronGliaNuclei()

# Saving off the Decomposition Table

In [ ]:
col_to_keep = ["n_vertices",           
"n_faces",              
"n_error_limbs",        
"n_same_soma_multi_touching_limbs",
"n_multi_soma_touching_limbs",
"n_somas",
"n_limbs",
"n_branches",
"max_limb_n_branches",
"skeletal_length",
"max_limb_skeletal_length",
"median_branch_length",
"width_median",
"width_no_spine_median",
"width_90_perc",
"width_no_spine_90_perc",
"n_spines",
"spine_density",
"spines_per_branch",
"skeletal_length_eligible",
"n_spine_eligible_branches",
"spine_density_eligible",
"spines_per_branch_eligible",
"total_spine_volume",
"spine_volume_median",
"spine_volume_density",
"spine_volume_density_eligible",
"spine_volume_per_branch_eligible",
"run_time"]

In [ ]:
decomp_saved_data = minnie.Decomposition.proj(*col_to_keep).fetch(as_dict=True)
decomp_saved_data

In [ ]:
new_data_frame

In [ ]:
new_data_frame = pd.DataFrame.from_dict(decomp_saved_data)
su.compressed_pickle(new_data_frame,"Decomposition_saved_1_11")

# Inspecting Decomposition

In [ ]:
curr_table = (schema.jobs & "table_name='__decomposition'") & "status='error'"#.delete()
curr_table

In [ ]:
(minnie.NeuronGliaNuclei() & "n_glia_faces>0").proj()

In [ ]:
minnie.Decomposition() #& (minnie.NeuronGliaNuclei() & "n_glia_faces>0").proj()

In [ ]:
segment_id = 864691135065018692

In [ ]:
neuron_obj = (minnie.Decomposition & dict(segment_id=segment_id)).fetch1("decomposition")

nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

# Inspecting Decomposition Split

In [6]:
import numpy as np

In [10]:
err_table = (schema.jobs & "table_name='__decomposition_split'")
err_messages = (err_table).fetch("error_message")
unique_errors,unique_counts = np.unique(err_messages,return_counts=True)
high_frequency_errors = np.where(unique_counts>20)[0]

for h_e in high_frequency_errors:
    print(f"{unique_counts[h_e]} : {unique_errors[h_e]}")

49 : 
107 : Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)
39 : IndexError: list index out of range
29 : IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
33 : SystemExit: SIGTERM received


In [11]:
du.restrict_jobs_table_by_error_substring(err_table,
                                         message_in_error="after loop in directed concept graph, not all nodes have incoming edges")

Found 107 matching entries with substring after loop in directed concept graph, not all nodes have incoming edges


table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_split,01c74daca008117152a53ca441a71306,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.144,at-node17,1,112052,2021-02-03 07:01:44
__decomposition_split,03333b84aebe25314db2b2eef70f6e2e,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.147,at-node20,1,112086,2021-02-03 04:15:53
__decomposition_split,0526605b148dd5629d420781702b32f7,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.81,at-compute003,1,112252,2021-02-03 04:59:41
__decomposition_split,08d49c4fcbe70799ca7104b2f3b9f93f,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.132,at-node5,1,112053,2021-02-03 03:59:45
__decomposition_split,0b955490bff8a69e1f3b40c1402b9357,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.132,at-node5,1,111972,2021-02-03 10:40:31
__decomposition_split,11a0e7e03b3c2160733025bf29245e40,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.139,at-node12,1,112011,2021-02-03 04:47:39
__decomposition_split,14849e726edfcf115a1bc4401202b602,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.81,at-compute003,1,112253,2021-02-03 05:57:46
__decomposition_split,1825c76d0b45d1c0c26a20201d6871a9,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.66.4.64,jr-compute001,1,112207,2021-02-03 04:39:58
__decomposition_split,187a97d705efcfb407d4d9a1adcc24a5,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.154,at-node27,1,112953,2021-02-03 15:01:00
__decomposition_split,1bf5bbaa622f3200d1ac808f69c31e7c,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.160,at-node33,1,112076,2021-02-03 06:58:57


In [12]:
specific_error = (err_table & "key_hash='187a97d705efcfb407d4d9a1adcc24a5'")
specific_error

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_split,187a97d705efcfb407d4d9a1adcc24a5,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.154,at-node27,1,112953,2021-02-03 15:01:00


In [13]:
print(specific_error.fetch1("error_stack"))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "Run_6_DecompositionSplit.py", line 252, in make
    return_error_info=True
  File "/meshAfterParty/proofreading_utils.py", line 1680, in split_neuron
    nru.clean_neuron_all_concept_network_data(split_neuron_obj)
  File "/meshAfterParty/neuron_utils.py", line 4111, in clean_neuron_all_concept_network_data
    starting_node=st_node)
  File "/meshAfterParty/neuron.py", line 870, in set_concept_network_directional
    **kwargs)
  File "/meshAfterParty/neuron.py", line 706, in convert_concept_network_to_directional
    suppress_disconnected_errors =suppress_disconnected_errors)
  File "/meshAfterParty/neuron_utils.py", line 597, in convert_concept_network_to_directional
    raise Exception("after loop in directed concept graph, not all nodes have incoming edges (except starter node)")
Exception: after loop in directed concept graph

In [14]:
print(specific_error.fetch1("key"))

{'segment_id': 864691135883900016, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 5, 'index': 0}


In [16]:
neur_obj = (minnie.Decomposition() & dict(segment_id=864691135883900016)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [17]:
specific_error.delete()

# Spine Recalculation

In [ ]:
minnie.DecompositionSplit.describe()

In [ ]:
len(minnie.SpineRecalculation())

In [ ]:
schema.jobs & "table_name='__spine_recalculation'"

In [ ]:
(schema.jobs & "table_name='__spine_recalculation'" & "status='error'")

# Inspeting the Auto Proofreading

In [40]:
len(minnie.AutoProofreadNeurons())

76696

In [15]:
import numpy as np

In [21]:
[j,3*j,2*j]

[1j, 3j, 2j]

In [24]:
np.flip([j,3*j,2*j])

array([0.+2.j, 0.+3.j, 0.+1.j])

In [25]:
j = np.complex(0,1)
roots = np.roots(np.flip([j,3*j,2*j]))

In [26]:
np.poly(roots)

array([1. , 1.5, 0.5])

In [32]:
len(schema.jobs & "table_name = '__auto_proofread_neurons'" & "status='error'")#.delete()

3444

In [ ]:
curr_table = (schema.jobs & "table_name = '__auto_proofread_neurons'" & "status='error'")#.delete()
curr_table_errors = du.restrict_jobs_table_by_error_substring(curr_table,"skeletal_length")

In [ ]:
print((schema.jobs & "table_name = '__auto_proofread_neurons'" & 'key_hash="5f108f5f1ef6e72e98b8796389e124e2"').fetch1("error_stack"))

In [ ]:
presyn_table = minnie.SynapseProofread & "synapse_type='presyn'"
postsyn_table = minnie.SynapseProofread & "synapse_type='postsyn'"

In [ ]:
presyn_table_renamed = presyn_table.proj(presyn="segment_id",
                                         presyn_split_index = "split_index",
                                         presyn_nucleus="nucleus_id",
                                         presyn_label="synapse_type")

postsyn_table_renamed = postsyn_table.proj(postsyn="segment_id",
                                           postsyn_split_index="split_inadex",
                                            postsyn_nucleus="nucleus_id",
                                           postsyn_label="synapse_type")

direct_conn_with_labels = presyn_table_renamed * postsyn_table_renamed
direct_conn_with_labels

In [ ]:
du.m65mat.NucleusSynapse.describe()

In [ ]:
import datajoint as dj
just_synapses_id_and_size = dj.U("synapse_id","synapse_size") & du.m65mat.NucleusSynapse.proj("synapse_size")

# want to get the synapse table as well
direct_conn_with_labels_and_sizes = just_synapses_id_and_size * direct_conn_with_labels
direct_conn_with_labels_and_sizes

In [ ]:
synapse_direct_connections = du.m65mat.NucleusSynapse * direct_conn_with_labels
synapse_direct_connections

In [ ]:
minnie.AutoProofreadNeurons() & "cell_type_predicted='excitatory'" & "spine_category='sparsely_spined'" & "n_branches>4"

In [ ]:
minnie.AutoProofreadNeurons() & "cell_type_predicted='inhibitory'"